In [2]:
ann_file="A01.ann"
txt_file="A01.txt"

In [3]:
import nltk

def read_ann_file(ann_file_path):
    annotations = []
    
    with open(ann_file_path, 'r') as file:
        for line in file:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                # Entity annotation (e.g., T1 Person 0 4 John)
                annotation_id, annotation_details = parts
                entity_type, start, end, text = annotation_details.split()
                annotations.append({
                    "id": annotation_id,
                    "type": entity_type,
                    "start": int(start),
                    "end": int(end),
                    "text": text
                })
            elif len(parts) == 3:
                # Relation annotation (e.g., R1 Located_in Arg1:T1 Arg2:T2)
                annotation_id, relation_type, args = parts
                arguments = args.split()
                annotations.append({
                    "id": annotation_id,
                    "relation_type": relation_type,
                    "arguments": arguments
                })
    
    return annotations


In [ ]:
annotations=read_ann_file(ann_file)


In [4]:
def extract_annotated_text(txt_file_path, annotations):
    with open(txt_file_path, 'r') as txt_file:
        text = txt_file.read()

    annotated_text = []
    
    for annotation in annotations:
        if "text" in annotation:
            start = annotation["start"]
            end = annotation["end"]
            annotated_text.append({
                "type": annotation["type"],
                "text": text[start:end]
            })
    
    return annotated_text
